# 19 - Metodos magicos: disenar objetos que se comportan como tipos nativos

## Objetivos de aprendizaje

En esta sesion aprenderas a:

1. Entender que son los metodos magicos (`__dunder__`) y cuando conviene implementarlos.
2. Diferenciar `__repr__` y `__str__` para depuracion vs experiencia de usuario.
3. Disenar semantica consistente para verdad logica (`__bool__`) y longitud (`__len__`).
4. Implementar igualdad (`__eq__`) y hash (`__hash__`) sin romper diccionarios ni sets.
5. Definir orden (`__lt__`, `__le__`, etc.) con criterio de dominio y pruebas.
6. Sobrecargar operadores (`__add__`, `__radd__`, `__iadd__`) evitando sorpresas.
7. Exponer colecciones personalizadas con `__getitem__`, `__iter__` y `__contains__`.
8. Usar `__call__` para objetos configurables que actuan como funciones.
9. Identificar errores frecuentes al modelar el protocolo de datos de Python.
10. Practicar decisiones de diseno en ejercicios que priorizan razonamiento tecnico.


## 1. Problema que resuelven los metodos magicos

Una clase puede "funcionar" con metodos normales, pero sentirse torpe al usarla.
Ejemplos de friccion:

- imprimir objetos devuelve algo poco util,
- no puedes usar `len(obj)` aunque el objeto tiene elementos,
- no puedes comparar, ordenar ni sumar objetos de forma natural,
- no puedes integrarlos bien con `for`, `in`, `sorted`, `set`, etc.

Los metodos magicos conectan tu clase con el ecosistema de Python.
La meta no es "usar dunder por moda", sino ofrecer una API que se lea natural.


In [ ]:
# Sin metodos magicos: la clase funciona, pero la experiencia es pobre.

class BolsaBasica:
    def __init__(self, elementos):
        self.elementos = list(elementos)

bolsa = BolsaBasica(["lapiz", "cuaderno", "usb"])
print(bolsa)                 # Representacion poco informativa
# print(len(bolsa))          # TypeError: no define __len__
# print("usb" in bolsa)      # TypeError: no define __contains__ ni __iter__


## 2. Modelo mental: Python invoca protocolos, no "magia"

Cuando escribes `len(x)`, Python llama internamente `x.__len__()`.
Cuando escribes `a + b`, Python intenta `a.__add__(b)`.
Cuando escribes `for item in x`, busca un protocolo de iteracion.

Puntos clave:

1. No llamas estos metodos de forma directa en uso normal.
2. Cada dunder representa un contrato de comportamiento.
3. Si implementas el contrato a medias, creas bugs sutiles.

Por eso conviene pensar primero la semantica de dominio y luego codificar.


In [ ]:
class Agenda:
    def __init__(self, contactos):
        self._contactos = list(contactos)

    def __len__(self):
        return len(self._contactos)

    def __iter__(self):
        return iter(self._contactos)

agenda = Agenda(["Ana", "Luis", "Marta"])
print(len(agenda))
for nombre in agenda:
    print(nombre)


## 3. `__repr__` vs `__str__`: depurar bien y comunicar mejor

`__repr__`:

- orientado a desarrolladores,
- debe ser claro, sin ambiguedad,
- idealmente util para reconstruir estado.

`__str__`:

- orientado a usuarios finales,
- mas legible y narrativo,
- puede omitir detalles tecnicos.

Regla practica: implementa al menos `__repr__`. Si hay interfaz humana, agrega `__str__`.


In [ ]:
class Pedido:
    def __init__(self, folio, total):
        if total < 0:
            raise ValueError("Total invalido")
        self.folio = folio
        self.total = total

    def __repr__(self):
        return f"Pedido(folio={self.folio!r}, total={self.total!r})"

    def __str__(self):
        return f"Pedido {self.folio} por ${self.total:.2f}"

p = Pedido("A-102", 259.9)
print(repr(p))
print(str(p))
print([p])  # Las colecciones usan repr de cada elemento


## 4. `__len__` y `__bool__`: verdad logica consistente

Python evalua un objeto como falso si:

1. define `__bool__` y retorna `False`, o
2. no define `__bool__`, pero `__len__` retorna 0.

Diseno recomendado:

- usa `__len__` cuando modelas una coleccion,
- define `__bool__` solo si tu nocion de "vacio" no depende de longitud,
- evita reglas sorpresivas (ejemplo: objeto con elementos que evalua False).


In [ ]:
class Carrito:
    def __init__(self):
        self._items = []

    def agregar(self, nombre):
        self._items.append(nombre)

    def __len__(self):
        return len(self._items)

carrito = Carrito()
print(bool(carrito), len(carrito))
carrito.agregar("cafe")
print(bool(carrito), len(carrito))


## 5. `__eq__` y `__hash__`: igualdad correcta sin romper estructuras hash

Si dos objetos representan la misma identidad de dominio, `==` deberia reflejarlo.
Pero cuidado: si redefiniste `__eq__`, Python puede deshabilitar `__hash__` para protegerte.

Contrato importante:

1. Si `a == b`, entonces `hash(a) == hash(b)`.
2. Objetos usados como clave deben ser inmutables en los campos que participan en hash.
3. Si no hay inmutabilidad clara, evita `__hash__`.


In [ ]:
class SKU:
    def __init__(self, codigo):
        self.codigo = codigo

    def __eq__(self, other):
        if not isinstance(other, SKU):
            return NotImplemented
        return self.codigo == other.codigo

    def __hash__(self):
        return hash(self.codigo)

s1 = SKU("ABC-001")
s2 = SKU("ABC-001")
print(s1 == s2)
print({s1, s2})  # Solo un elemento porque son equivalentes


## 6. Comparaciones y orden: `__lt__` con criterio de negocio

`sorted()` y `list.sort()` necesitan poder comparar elementos.
Si tu clase define orden, ese orden debe ser coherente y explicable.

Recomendaciones:

- define el criterio principal de forma explicita,
- para comparaciones no soportadas, retorna `NotImplemented`,
- no mezcles criterios incompatibles sin documentarlos.


In [ ]:
from functools import total_ordering

@total_ordering
class TicketSoporte:
    def __init__(self, prioridad, creado_en):
        self.prioridad = prioridad     # 1 es mas urgente que 5
        self.creado_en = creado_en     # menor valor = mas antiguo

    def __eq__(self, other):
        if not isinstance(other, TicketSoporte):
            return NotImplemented
        return (self.prioridad, self.creado_en) == (other.prioridad, other.creado_en)

    def __lt__(self, other):
        if not isinstance(other, TicketSoporte):
            return NotImplemented
        return (self.prioridad, self.creado_en) < (other.prioridad, other.creado_en)

cola = [
    TicketSoporte(3, 15),
    TicketSoporte(1, 30),
    TicketSoporte(1, 12),
]
print([(t.prioridad, t.creado_en) for t in sorted(cola)])


## 7. Operadores: `__add__`, `__radd__`, `__iadd__` con semantica clara

Sobrecargar operadores puede mejorar legibilidad o arruinarla.

Buenas practicas:

1. Solo sobrecarga si el operador tiene sentido natural en tu dominio.
2. Si no sabes combinar tipos distintos, retorna `NotImplemented`.
3. Define si la operacion crea objeto nuevo (`__add__`) o muta (`__iadd__`).
4. Evita "magia oculta" que sorprenda al equipo.


In [ ]:
class Dinero:
    def __init__(self, monto, moneda="MXN"):
        self.monto = float(monto)
        self.moneda = moneda

    def __repr__(self):
        return f"Dinero(monto={self.monto}, moneda={self.moneda!r})"

    def __add__(self, other):
        if not isinstance(other, Dinero):
            return NotImplemented
        if self.moneda != other.moneda:
            raise ValueError("No puedes sumar monedas distintas")
        return Dinero(self.monto + other.monto, self.moneda)

    def __radd__(self, other):
        if other == 0:
            return self
        return self.__add__(other)

saldo = Dinero(150) + Dinero(50)
print(saldo)
print(sum([Dinero(100), Dinero(30), Dinero(20)], Dinero(0)))


## 8. Clases que se sienten como colecciones

Con estos metodos puedes integrarte al ecosistema de colecciones:

- `__getitem__`: acceso por indice o llave,
- `__setitem__`: asignacion por indice o llave,
- `__contains__`: operador `in`,
- `__iter__`: iteracion con `for`.

Si tu clase representa datos indexables, vale mucho la pena implementar este protocolo.


In [ ]:
class InventarioSimple:
    def __init__(self):
        self._stock = {}

    def __getitem__(self, sku):
        return self._stock.get(sku, 0)

    def __setitem__(self, sku, cantidad):
        if cantidad < 0:
            raise ValueError("Stock invalido")
        self._stock[sku] = cantidad

    def __contains__(self, sku):
        return sku in self._stock

    def __iter__(self):
        return iter(self._stock.items())

inv = InventarioSimple()
inv["A"] = 5
inv["B"] = 0
print(inv["A"], "A" in inv, "C" in inv)
for sku, cantidad in inv:
    print(sku, cantidad)


## 9. `__call__`: objetos configurables que actuan como funciones

`__call__` permite que una instancia sea invocable con parentesis.
Es util cuando quieres combinar:

- configuracion inicial,
- estado acumulado,
- y una interfaz simple de uso.

Piensa en validadores, filtros, politicas de negocio o pipelines.


In [ ]:
class ReglaMinimo:
    def __init__(self, minimo):
        self.minimo = minimo

    def __call__(self, valor):
        return valor >= self.minimo

es_mayor_edad = ReglaMinimo(18)
print(es_mayor_edad(16))
print(es_mayor_edad(21))


## 10. Caso real pequeno: clase de dominio con comportamiento natural

Escenario: quieres modelar rangos horarios para una agenda.
Necesitas comparar, imprimir y validar pertenencia de minutos.

Decision de diseno:

1. `__contains__` para `minuto in rango`.
2. `__len__` para saber duracion.
3. `__repr__` para depurar incidencias.

La clase queda facil de leer para cualquier persona del equipo.


In [ ]:
class RangoMinutos:
    def __init__(self, inicio, fin):
        if not (0 <= inicio < fin <= 24 * 60):
            raise ValueError("Rango invalido")
        self.inicio = inicio
        self.fin = fin

    def __repr__(self):
        return f"RangoMinutos(inicio={self.inicio}, fin={self.fin})"

    def __len__(self):
        return self.fin - self.inicio

    def __contains__(self, minuto):
        return self.inicio <= minuto < self.fin

bloque = RangoMinutos(9 * 60, 11 * 60)
print(bloque, len(bloque))
print(10 * 60 in bloque, 12 * 60 in bloque)


## 11. Errores comunes con metodos magicos

1. Implementar dunder "porque se ve avanzado", sin necesidad real.
2. Definir `__eq__` y olvidar `__hash__` cuando la clase ira a `set` o `dict`.
3. Sobrecargar operadores con semantica confusa (ejemplo: `+` que borra datos).
4. Romper expectativas de verdad logica (`bool(obj)`) respecto a su estado.
5. Hacer comparaciones parciales que no son transitivas.
6. Capturar errores y devolver valores silenciosos que ocultan bugs.
7. No cubrir contratos con pruebas de comportamiento.


## 12. Checklist rapido de diseno

Antes de agregar un metodo magico, responde:

1. Que operacion de Python quiero habilitar y por que?
2. Mi implementacion respeta expectativas de tipos nativos?
3. La semantica esta alineada con el dominio del problema?
4. Este metodo introduce ambiguedad para el equipo?
5. Tengo pruebas que validen casos normales y bordes?

Si no puedes justificarlo, probablemente no necesitas ese dunder.


## 13. Ejercicios de pensamiento y practica real

La meta no es copiar codigo.
La meta es tomar decisiones de diseno con argumentos tecnicos y validarlas con pruebas.

Recomendacion: escribe primero supuestos e invariantes, despues implementa.


### Ejercicio 1: contrato minimo para un tipo de dominio

**Escenario**: sistema academico con una clase `Calificacion`.

**Tarea**:
1. Decide que metodos magicos SI deberia tener (`__repr__`, `__str__`, `__eq__`, otros).
2. Justifica cuales NO deberia tener y por que.
3. Define dos invariantes del dominio.
4. Implementa una version minima coherente.


In [ ]:
class Calificacion:
    # Implementa aqui tu propuesta.
    # Pista: decide si conviene tratarla como valor inmutable.
    pass


### Ejercicio 2: `__repr__` util para depuracion real

Analiza esta clase y mejora su representacion.

**Tarea**:
1. Detecta por que el `repr` actual dificulta debug.
2. Reescribe `__repr__` con informacion critica.
3. Agrega un `__str__` orientado a usuario.
4. Muestra un ejemplo donde la mejora ahorre tiempo de diagnostico.


In [ ]:
class SesionUsuario:
    def __init__(self, usuario_id, ip, activa):
        self.usuario_id = usuario_id
        self.ip = ip
        self.activa = activa

    def __repr__(self):
        return "SesionUsuario()"  # Mejorar


### Ejercicio 3: igualdad de identidad vs igualdad de estado

**Escenario**: clase `Empleado` con `id_empleado`, `nombre`, `salario`.

**Tarea**:
1. Elige criterio de igualdad para tu negocio y argumentalo.
2. Implementa `__eq__` y, si aplica, `__hash__`.
3. Demuestra con ejemplos como cambia el comportamiento en `set`.
4. Explica riesgos de elegir mal el criterio.


In [ ]:
class Empleado:
    def __init__(self, id_empleado, nombre, salario):
        self.id_empleado = id_empleado
        self.nombre = nombre
        self.salario = salario

    # Implementa aqui igualdad y hash segun tu decision.


### Ejercicio 4: bug sutil de hash por mutabilidad

La siguiente clase parece correcta, pero puede romper diccionarios.

**Tarea**:
1. Reproduce el bug.
2. Explica la causa exacta.
3. Propone dos soluciones con tradeoffs.
4. Implementa una solucion segura.


In [ ]:
class ProductoHashRiesgoso:
    def __init__(self, codigo, precio):
        self.codigo = codigo
        self.precio = precio

    def __eq__(self, other):
        if not isinstance(other, ProductoHashRiesgoso):
            return NotImplemented
        return (self.codigo, self.precio) == (other.codigo, other.precio)

    def __hash__(self):
        return hash((self.codigo, self.precio))

# Muta precio despues de usarlo como clave y observa el problema.


### Ejercicio 5: orden total para una cola de prioridad

**Escenario**: tickets con severidad, impacto y tiempo de creacion.

**Tarea**:
1. Define una tupla de ordenamiento defendible.
2. Implementa comparaciones minimas (`__eq__`, `__lt__`) y usa `@total_ordering`.
3. Verifica transitividad con al menos 3 objetos.
4. Explica que pasaria si cambias el criterio en produccion.


In [ ]:
from functools import total_ordering

@total_ordering
class Ticket:
    def __init__(self, severidad, impacto, creado_en):
        self.severidad = severidad
        self.impacto = impacto
        self.creado_en = creado_en

    # Implementa __eq__ y __lt__.


### Ejercicio 6: coleccion personalizada con slicing

**Escenario**: historial de transacciones con lectura por indice y por rango.

**Tarea**:
1. Implementa `__getitem__` para soportar indice y slice.
2. Define que error lanzar en indices invalidos.
3. Agrega `__len__` y `__iter__`.
4. Justifica por que tu API se parece a una lista y donde difiere.


In [ ]:
class HistorialTransacciones:
    def __init__(self, transacciones):
        self._transacciones = list(transacciones)

    # Implementa __getitem__, __len__ y __iter__.


### Ejercicio 7: `__call__` para reglas configurables

**Escenario**: motor antifraude con reglas que pueden encadenarse.

**Tarea**:
1. Crea al menos 3 reglas invocables (`__call__`).
2. Cada regla recibe una compra y devuelve `(ok, motivo)`.
3. Dise?a un motor que combine reglas sin conocer clases concretas.
4. Argumenta como agregarias una regla nueva sin tocar el motor.


In [ ]:
class MotorAntifraude:
    def __init__(self):
        self._reglas = []

    def registrar(self, regla):
        self._reglas.append(regla)

    def evaluar(self, compra):
        resultados = []
        for regla in self._reglas:
            resultados.append(regla(compra))
        return resultados

# Implementa reglas concretas e integra el motor.


### Ejercicio 8: sobrecarga de operadores sin ambiguedad

**Escenario**: clase `Vector2D` para calculos basicos.

**Tarea**:
1. Implementa `__add__` y `__sub__` entre vectores.
2. Decide si soportaras multiplicacion por escalar y donde (`__mul__`, `__rmul__`).
3. Retorna `NotImplemented` en tipos no compatibles.
4. Explica una decision para evitar malentendidos matematicos.


In [ ]:
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Vector2D(x={self.x}, y={self.y})"

    # Implementa operadores con semantica clara.


### Ejercicio 9: diseno de API para objetos "truthy/falsy"

**Escenario**: clase `ConexionBD` con estados `conectado`, `latencia_ms`, `errores`.

**Tarea**:
1. Decide si conviene implementar `__bool__`.
2. Define criterio explicito de "conexion usable".
3. Implementa la clase y muestra ejemplos limite.
4. Explica riesgos de ocultar demasiada logica en `if conexion:`.


In [ ]:
class ConexionBD:
    def __init__(self, conectado, latencia_ms, errores):
        self.conectado = conectado
        self.latencia_ms = latencia_ms
        self.errores = errores

    # Decide e implementa __bool__ (o justifica por que no).


### Ejercicio 10: mini proyecto integrador

**Reto**: construir un sistema de carrito para e-commerce con objetos de dominio propios.

Debe incluir:
1. `Producto` con representacion util,
2. `LineaCarrito` comparable u ordenable por criterio que definas,
3. `Carrito` iterable y con `len`,
4. reglas de descuento invocables (`__call__`),
5. soporte de suma de subtotales con operador o metodo bien justificado.

**Entrega esperada**:
- implementacion minima funcional,
- pruebas con `assert` de contratos importantes,
- explicacion de tradeoffs y deuda tecnica consciente.


In [ ]:
# Implementa aqui tu solucion.
# Consejo: empieza por invariantes y casos borde.


## 14. Resumen de conceptos clave

1. Los metodos magicos conectan tu clase con operaciones nativas de Python.
2. Cada dunder debe responder a una necesidad real del dominio.
3. `__repr__` mejora depuracion; `__str__` mejora comunicacion con usuarios.
4. Igualdad y hash exigen consistencia e idealmente inmutabilidad.
5. Orden y operadores deben ser predecibles y defendibles.
6. Protocolos de coleccion (`len`, iteracion, indexado, pertenencia) reducen friccion de uso.
7. `__call__` permite objetos configurables con interfaz simple.
8. Buen diseno se demuestra en claridad de API, pruebas y facilidad de evolucion.


## 15. Reto final opcional

Disena un mini motor de recomendaciones de estudio con:

- `Tema` y `Recurso` como objetos de dominio,
- reglas ponderadas para priorizar recursos,
- comparacion y orden de recomendaciones,
- interfaz de evaluacion invocable con `__call__`.

Objetivo:

1. elegir metodos magicos con criterio (no por cantidad),
2. justificar cada decision de semantica,
3. demostrar con pruebas que la API es natural y estable.
